# Scraping Walmart Coolers with Selenium 2019

## Step 1: import the libraries 

In [202]:
import csv
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

## Step 2: load the web driver and the page

In [203]:
# import the webdriver, chrome driver is recommended
#https://sites.google.com/a/chromium.org/chromedriver/downloads
#driver = webdriver.Chrome("/Users/junho/Google Drive/Capstone Project/Scraper/chromedriver")
driver = webdriver.Chrome("/Users/junhong/Google Drive/Spring 2020/Capstone Project/Scraper/chromedriver2")

# insert Walmart cooler search results 
driver.get("https://www.walmart.com/search/?grid=false&query=cooler")

In [204]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

## Step3: save brand, item names and price

In [161]:
#get price
#driver.find_element_by_xpath('//*[@id="searchProductResult"]/div/div[1]/div/div/div[2]/div[2]/div[2]/span/span').text
#container[0].find_element_by_class_name('search-result-product-rating').text
#container[4].find_element_by_class_name('prod-desc-priceUI-1').text
#container[0].find_element_by_class_name('search-result-product-rating').text
#num_ratings = container[4].find_element_by_class_name('search-result-product-rating').text
#num_ratings

In [206]:
%%time
product_list = []
#Get number of item pages
num_page = int(driver.find_element_by_xpath('//*[@id="mainSearchContent"]/div[3]/div[2]/ul/li[8]/a').text)
for i in range(num_page):    
    container = driver.find_elements_by_class_name('search-result-listview-item.Grid')
    #This will get number of items in a page
    num_items = len(container)
    for j in range(num_items):
        container = driver.find_elements_by_class_name('search-result-listview-item.Grid')
        product_name = container[j].find_element_by_class_name('search-result-product-title.listview').text.split("\n")[1]
        
        #Get price, sometimes price is not avaliable so we have empty string instead
        try:
            product_price_low = container[j].find_element_by_class_name('price-main-block').text.split("\n")[0]
        except Exception:
            product_price_low = ''
        try:
            product_price_high = container[j].find_element_by_class_name('price-main-block').text.split("\n")[-1]
        except Exception:
            product_price_high = ''
        product_num_rating = container[j].find_element_by_class_name('search-result-product-rating').text
        product_num_rating = re.findall(r'.+on(\d+)rev.+',product_num_rating)[0]
        product_rating = container[j].find_element_by_class_name('search-result-product-rating').text
        product_rating = re.findall(r'.+:(.+)out.+',product_rating)[0] 
        
        #Sometimes there is no description
        try:
            product_description = container[j].find_element_by_class_name('prod-desc-priceUI-1').text
        except Exception:
            product_description = ''
        #Append all information to a list
        product_list.append([product_name,product_price_low,product_price_high,product_num_rating, product_rating, product_description])
    
    #the "next page" button has a different XPath on first page
    if i == 0:
        driver.find_element_by_xpath('//*[@id="mainSearchContent"]/div[3]/div[2]/button').click()
        time.sleep(10)
    else if i != num_page-1:
        driver.find_element_by_xpath('//*[@id="mainSearchContent"]/div[3]/div[2]/button[2]').click()
        time.sleep(10)
    else:
        pass

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mainSearchContent"]/div[3]/div[2]/button[2]"}
  (Session info: chrome=78.0.3904.108)


In [207]:
df = pd.DataFrame(product_list, columns = ['name','price_low','price_high','rating_num','rating','description'])

In [208]:
df

,name,price_low,price_high,rating_num,rating,description
0,Coleman 50-Quart Marine Cooler,$30.99,$30.99,39,3.8,The Coleman Xtreme Marine 50-Quart Cooler feat...
1,Coleman 50Qt Xtreme Cooler,$24.99,$24.99,61,3.4,Keeps the Ice! up to 5 days at temperatures up...
2,"Ozark Trail Premium 36 Can Cooler, White",$29.00,$49.95,114,2.6,Made from easy to clean 600D-coated material\n...
3,Coleman 48 qt Inland Performance Series Marine...,$21.99,$21.99,27,4.4,UVGuard material\nUp to 3 days at temperatures...
4,"Ozark Trail 30 Can Leak Tight Cooler, Vector Camo",$29.00,$29.00,0,0,
...,...,...,...,...,...,...
991,Inflatable Treasure Chest Cooler,$24.38,$24.38,4,1.8,Has look of a wooden chest with gold accents\n...
992,Avalon Self Clean Bottleless Hot/Cold Water Co...,$203.08,$203.08,80,4.7,"? Hot, cold, and room temperature water output..."
993,IGLOO Legend 1/2 Gallon Jug,$7.99,$32.34,15,4.1,Ultratherm insulation provides extended ice re...
994,evaporative air cooler,$95.18,$95.18,45,3.2,evaporative air cooler


In [209]:
driver.close()

In [210]:
df.to_csv('Walmart_Cooler.csv',index=False)